In [2]:
import requests
import pandas as pd

In [6]:
api_key = "your_api_key"
channel_id = "your_channel_id"

sub_channels_url = f"https://www.googleapis.com/youtube/v3/subscriptions?part=snippet&channelId={channel_id}&key={api_key}&maxResults=200"

In [25]:
sub_channels_response = requests.get(sub_channels_url).json()

In [32]:
def populate_subscribed_channels(sub_channels):
    df = pd.DataFrame(columns = ["channel_id", "channel_title", "channel_description", "creation_date"])
    
    for i in range(len(sub_channels)):
        new_row = {
            "channel_id": sub_channels[i]["snippet"]["resourceId"]["channelId"],
            "channel_title": sub_channels[i]["snippet"]["title"],
            "channel_description": sub_channels[i]["snippet"]["description"],
            "creation_date": sub_channels[i]["snippet"]["publishedAt"]
        }
        df = df.append(new_row, ignore_index=True)
        
        df["creation_date"] = df["creation_date"].str.replace("T", " ").str.replace("Z", "")
    return df

In [33]:
sub_channels = populate_subscribed_channels(sub_channels_response["items"])

In [34]:
print(sub_channels.head())

                 channel_id      channel_title  \
0  UC1GdNuXC7byr3d-b1m7er0Q         Le Musclay   
1  UC2avy3Pwc3yVtt-tPKJn33A  ma ferme autonome   
2  UC4EDwdyja1R7-dOKa7J4cGw     La maison en A   
3  UC4peNo-31r7fhQZgu0_JRIA         Sheshounet   
4  UC7D9Q0WzEejlgdXywAgz79A  Réussite Critique   

                                 channel_description  \
0  Musculation, nutrition, saupoudré de clashs et...   
1  Produire sa nourriture et son énergie. S'appro...   
2  Tout est possible. C’est ce concept que j’ai v...   
3  Je suis Sheshounet et le contenu que je propos...   
4  Mahyar, auteur, justicier, mercenaire. Au serv...   

                creation_date  
0         2015-01-28 19:50:13  
1         2017-10-16 19:58:15  
2  2021-05-24 07:42:31.383795  
3    2018-12-27 20:03:38.5395  
4         2016-01-17 09:49:20  


In [37]:
def get_channel_statistics(channel_id):
    channel_stat_url = f"https://www.googleapis.com/youtube/v3/channels?part=statistics&id={channel_id}&key={api_key}"
    channel_stat_response = requests.get(channel_stat_url).json()
    
    response_stats = channel_stat_response["items"][0]["statistics"]
    
    channel_stats = {
        "view_count": response_stats["viewCount"],
        "video_count": response_stats["videoCount"],
        "number_subscriber": response_stats["subscriberCount"]
    }
    return channel_stats

def populate_channels_statistics(channels_id):
    df = pd.DataFrame(columns = ["view_count", "video_count", "number_subscriber"])
    
    for channel in channels_id:
        df = df.append(get_channel_statistics(channel), ignore_index = True)
    return df

In [40]:
sub_channels_stats = populate_channels_statistics(sub_channels["channel_id"])
sub_channels = pd.concat([sub_channels, sub_channels_stats], axis = 1)

In [42]:
print(sub_channels.head())

                 channel_id      channel_title  \
0  UC1GdNuXC7byr3d-b1m7er0Q         Le Musclay   
1  UC2avy3Pwc3yVtt-tPKJn33A  ma ferme autonome   
2  UC4EDwdyja1R7-dOKa7J4cGw     La maison en A   
3  UC4peNo-31r7fhQZgu0_JRIA         Sheshounet   
4  UC7D9Q0WzEejlgdXywAgz79A  Réussite Critique   

                                 channel_description  \
0  Musculation, nutrition, saupoudré de clashs et...   
1  Produire sa nourriture et son énergie. S'appro...   
2  Tout est possible. C’est ce concept que j’ai v...   
3  Je suis Sheshounet et le contenu que je propos...   
4  Mahyar, auteur, justicier, mercenaire. Au serv...   

                creation_date view_count video_count number_subscriber  
0         2015-01-28 19:50:13   26446870         100            206000  
1         2017-10-16 19:58:15   17326781          83            228000  
2  2021-05-24 07:42:31.383795     397014          34              8110  
3    2018-12-27 20:03:38.5395  135080039         196            840000

In [44]:
path_to_save = "path/to/project"
sub_channels.to_csv(path_to_save + "/sub_channels_data.csv", index=False)

In [27]:
def get_video_language(video_snippet):
    try:
        video_language = video_snippet["defaultLanguage"]
    except KeyError:
        try:
            video_language = video_snippet["defaultAudioLanguage"]
        except KeyError:
            video_language = ""
    return video_language


def get_video_tags(video_snippet):
    try:
        video_tags = "/".join(video_snippet["tags"])
    except KeyError:
        video_tags = ""
    return video_tags

def get_comments_count(video_stat):
    try:
        comments_count = video_stat["commentCount"]
    except KeyError:
        comments_count = ""
    return comments_count
        
def get_like_count(video_stat):
    try:
        like_count = video_stat["likeCount"]
    except KeyError:
        like_count = ""
    return like_count


def get_video_statistics(video_id):
    statistics_url = f"https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id={video_id}&key={api_key}"
    video_stat_response = requests.get(statistics_url).json()
    
    video_item = video_stat_response["items"][0]
    video_snippet = video_item["snippet"]
    video_stat = video_item["statistics"]
    
    # print(video_stat_response)
    
    video_row = {
        "video_id": video_item["id"],
        "channel_id": video_snippet["channelId"],
        "publication_date": video_snippet["publishedAt"],
        "video_language":  get_video_language(video_snippet),
        "video_title": video_snippet["title"],
        "video_description": video_snippet["description"],
        "video_tags": get_video_tags(video_snippet),
        "view_count": video_stat["viewCount"],
        "like_count": get_like_count(video_stat),
        "comment_count": get_comments_count(video_stat),
    }
    
    return video_row
    

def create_video_statistics(channel_id, max_video = 200):
    video_url = f"https://www.googleapis.com/youtube/v3/search?part=snippet,id&channelId={channel_id}&key={api_key}&maxResults={max_video}&order=date"
    video_response = requests.get(video_url).json()
    
    df = pd.DataFrame(columns = ["video_id", "channel_id", "publication_date", "video_language", "video_title", "video_description", "video_tags", "view_count", "like_count", "comment_count"])

    all_video_response = video_response["items"]
    
    for i in range(len(all_video_response)):
        if all_video_response[i]["id"]["kind"] == "youtube#video":
            current_id = all_video_response[i]["id"]["videoId"]
            # print(f"iteration {i} : {current_id} from {channel_id}")
            df = df.append(get_video_statistics(current_id), ignore_index = True)
    return df

In [31]:
for i in range(sub_channels.shape[0]):
    current_channel_id = sub_channels["channel_id"][i]
    current_channel_title = sub_channels["channel_title"][i]
    current_channel_title = current_channel_title.split("|")[0]

    print(f"current channel : {current_channel_title} - {current_channel_id}")

    channel_stat = create_video_statistics(current_channel_id)
    channel_stat.to_csv(path + f"/{current_channel_title}_videos_stat.csv")

current channel : piero 83 - UCDYVDDEHh-l_la4qREvLTyg
current channel : Wakademy  - UCEZAl0LQrixylf6jrktlabg
current channel : EthosLab - UCFKDEp9si4RmHFWJW1vYsMA
current channel : LeLapinTaquin - UCFqBcoewmW6w2bHqacxYJSw
current channel : Luke McLaughlin - UCHyxustj5PwgOJY0k--kkKw
current channel : tn DNA - UCIZPVRQAqg9LC-yD04WJseQ
current channel : Psyko Sauce - UCK1TdGB78zp6Rl6L4RK-hWg
current channel : Caljbeut - UCNM-UkIP1BL5jv9ZrN5JMCA
current channel : Neuro - UCOUtDNm-8Td8AaJvMLxujtg
current channel : Grian - UCR9Gcq0CMm6YgTzsDxAxjOQ
current channel : xisumavoid - UCU9pX8hKcrx06XfOB-VQLdw
current channel : Absol Vidéos - UCUR03ZSZlEvCO6EWwv4jU2w
current channel : Grand Angle - UCWtD8JN9hkxL5TJL_ktaNZA
current channel : La statistique expliquée à mon chat - UCWty1tzwZW_ZNSp5GVGteaA
current channel : Lucas Gouiffes - UCXJasFdYGWkB-7ObGJL4Ytg
current channel : Shawn Woods - UCYbru-MPO1xjes4FVn61JUQ
current channel : Valek - UCYdyGYM1aoLpAuCxsP86O1g
current channel : iskall85 - UCZ